In [ ]:
# imports
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import pprint

sys.path.append('../../modules/board_support_software/software/py/')
sys.path.append('../py/')
from module_includes import *

from pALPIDE import *
from communication import WishboneReadError, AddressMismatchError

import communication
import ru_eyescan
import testbench

In [ ]:
RUN_STANDALONE=True
config_file_path = script_path + '/../config/testbench_ibs.yml'
tb = testbench.configure_testbench(config_file_path=config_file_path,
                                   run_standalone=RUN_STANDALONE)
tb.cru.initialize()
tb.initialize_all_rdos()

In [ ]:
ch = Alpide(tb.rdo, chipid=0x00)
ch.reset()
ch.configure_dtu()

In [ ]:
ch.setreg_dtu_dacs(PLLDAC=8,DriverDAC=4,PreDAC=4)

In [ ]:
ch.setreg_dtu_test_1(TestEN=True,IntPatternEN=True,PrbsRate=0,Bypass8b10b=True)

In [ ]:
def pattern30(ch,pattern):
    ch.setreg_dtu_test_1(TestEN=True,IntPatternEN=False,PrbsRate=0,Bypass8b10b=True,BDIN8b10b0=pattern>>8&3,BDIN8b10b1=pattern>>18&3,BDIN8b10b2=pattern>>28&3)
    ch.setreg_dtu_test_2(DIN0=pattern&255,DIN1=pattern>>10&255)
    ch.setreg_dtu_test_3(DIN2=pattern>>20&255)
pattern30(ch,0b11101110001110111)

In [ ]:
ch.setreg_dtu_test_2()
ch.setreg_dtu_test_1(TestEN=True,IntPatternEN=False,PrbsRate=0,Bypass8b10b=True)

In [ ]:
ch.propagate_clock()

In [ ]:
## Setup
PllDac = 0x8
DriverDac= 0x8
PreDac = 0x8
SENSORS = [0] # Active Sensors
TRANSCEIVERS = [0] # Active Transceivers (Transeivers are mapped inversely? to sensors: ChipId0 => Tr8, ChipId8 -> Tr0)
USE_PRBS_SCAN = True # Run PRBS pattern on sensor during scan

EYESCAN_FILE = 'Test_eyescan_flp.csv'
EYESCAN_TRANSCEIVER = 0 # Transceiver to be used for the scan
HSTEPS = 16 # Number of horizontal points
VSTEPS = 16 # Nomber of vertical points
VERTICAL_RANGE = 3 # Voltage resolution per code: 00 -> 1.5mV, 01 -> 1.8mV, 10 -> 2.2mV, 11 -> 2.8mV
PRESCALE = 6 # Starting prescale. Max Nr. samples per point: 20 * 2**16 * 2**(1 + prescale)
BER = 8 # Maximum Prescale to scan to
NO_CENTER = True # Skip Center points (after successive 0 samples are found on this Prescale, following points are ignored)

tb.rdo.gth.set_transceivers(TRANSCEIVERS)
ch = Alpide(tb.rdo, chipid=0x0F)
ch.reset()
tb.setup_sensors(disable_manchester=0)
ch.setreg_dtu_dacs(PLLDAC=PllDac,DriverDAC=DriverDac,PreDAC=PreDac)

def check_errors():
    if USE_PRBS_SCAN:
        prbs_errors = tb.rdo.gth.read_prbs_counter(reset=True)
        all_errors = 0
        for cnt, link in zip(prbs_errors, tb.rdo.gth.transceivers):
            if cnt > 0:
                tb.logger.error(
                    "Link %d: %d PRBS Errors observed", link, cnt)
            all_errors += cnt
        tb.logger.info("PRBS run finished. Total Errors: %d", all_errors)
        return all_errors
    else:
        decode_errors = tb.rdo.datapathmon.read_counters(counters=['DECODE_ERROR_COUNT'])
        all_errors = 0
        for lane,count in decode_errors:
            if count['DECODE_ERROR_COUNT'] > 0:
                tb.logger.error("Link %d: %d Decoding Errors observed", link, cnt)
                all_errors += cnt
        return all_errors


tb.rdo.initialize()
tb.setup_readout()

if USE_PRBS_SCAN:
    tb.setup_prbs_test(SENSORS)

# Check prbs is setup properly
time.sleep(4)

assert check_errors() == 0, "Errors in Stream: Not set up?"

In [ ]:
transceiver = 0
tb.rdo.gth.set_transceivers([EYESCAN_TRANSCEIVER])
eyescan = ru_eyescan.EyeScanGth(tb.rdo.gth, vertical_range=VERTICAL_RANGE) # vertical range (division of voltage DAC)
eyescan.initialize()
eyescan._eye_scan_point_setup(offset_h=0,offset_v=0,prescale=0,ut_sign=0,readback=True)
eyescan._eye_scan_point_start()
time.sleep(0.3)
assert eyescan._is_eye_scan_point_done(), "Eyescan not yet done. Took too long"
assert eyescan._read_counts()[1]==0, "Not aligned, restart from initialize_readout"

In [ ]:
# Adaptive eye scanning with PRBS in background on all chips
tb.rdo.gth.set_transceivers(TRANSCEIVERS)
tb.rdo.datapathmon.reset_counters()
tb.rdo.gth.reset_prbs_counter()
start = time.time()

tb.rdo.gth.set_transceivers([EYESCAN_TRANSCEIVER])

# Adaptive Eye scanning
func = None
#func = bathtub_func
filename = EYESCAN_FILE
resume = False
verbose = False

data = eyescan.eye_scan_adaptive(v_steps = VSTEPS, 
                                 h_steps = HSTEPS,
                                 prescale = PRESCALE, 
                                 ber=BER, 
                                 resume = resume,
                                 output_file_name=filename, 
                                 verbose=verbose,
                                 func=func,
                                 no_center=NO_CENTER)

tb.rdo.gth.set_transceivers(TRANSCEIVERS)
errors = check_errors()
print(f"After {time.time()-start:.2f} s, {errors} errors")